# Custom Perceptron Model

In [1]:
# Import the necessary library
import numpy as np

# Set the random seed legacy
np.random.seed(523)
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
np.random.seed(523)

# Build the Perceptron Model
class Perceptron:
    
    def __init__(self, num_inputs, learning_rate=0.1):
        # Initialize the weight and learning rate
        self.weights = np.random.rand(num_inputs + 1)
        self.learning_rate = learning_rate
    
    # Define the first linear layer 
    def linear(self, inputs):
        Z = inputs @ self.weights[1:].T + + self.weights[0]
        return Z
    
    # Define the Heaviside Step function.
    def Heaviside_step_fn(self, z):
        if z >= 0:
            return 1
        else:
            return 0
        
    # Define the Prediction
    def predict(self, inputs):
        Z = self.linear(inputs)
        try:
            pred = []
            for z in Z:
                pred.append(self.Heaviside_step_fn(z))
        except:
            return self.Heaviside_step_fn(Z)
        return pred
    
    # Define the Hinge Loss function
    def hinge_loss(self, prediction, target):
        loss = np.maximum(0, 1 - prediction * target)
        return loss
    
    #Define training
    def train(self, inputs, target):
        prediction = self.predict(inputs)
        loss = self.hinge_loss(prediction, target)
        error = target * (loss > 0)
        self.weights[1:] += self.learning_rate * error * inputs
        self.weights[0]  += self.learning_rate * error
        
    # Fit the model
    def fit(self, X, y, num_epochs):
        for epoch in range(num_epochs):
            for inputs, target in zip(X, y):
                self.train(inputs, target)


# Load the Breast Cancer dataset
cancer = load_breast_cancer()


# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, test_size=0.2, random_state=23)


# Scale the input features to have zero mean and unit variance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize the Perceptron with the appropriate number of inputs
perceptron = Perceptron(num_inputs=X_train.shape[1])

# Train the Perceptron on the training data
perceptron.fit(X_train, y_train, num_epochs=100)

# Prediction
pred = perceptron.predict(X_test)

# Test the accuracy of the trained Perceptron on the testing data
accuracy = np.mean(pred == y_test)
print("Accuracy:", accuracy)

Accuracy: 0.9298245614035088


# Perceptron with PyTorch

In [2]:
# Import the necessary libraries
import torch
import torch.nn as nn
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Breast Cancer dataset
cancer = load_breast_cancer()

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, test_size=0.2, random_state=42)

# Scale the input features to have zero mean and unit variance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert the data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

# reshape the target tensor to match the predicted output tensor
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)


# Define the Perceptron model
class Perceptron(nn.Module):
    def __init__(self, num_inputs):
        super(Perceptron, self).__init__()
        self.linear = nn.Linear(num_inputs, 1)
    
    def forward(self, x):
        return torch.sigmoid(self.linear(x))

# Initialize the Perceptron with the appropriate number of inputs
perceptron = Perceptron(num_inputs=X_train.shape[1])


# Binary Cross Entropy / Log likelyhood
def Binary_Cross_Entropy(y_pred,Y):
    cost = torch.mean((-Y * torch.log(y_pred)) - ((1-Y) * torch.log(1-y_pred)))
    return cost

# Define optimizer
optimizer = torch.optim.SGD(perceptron.parameters(), lr=0.01)

# Train the Perceptron on the training data
num_epochs = 500
for epoch in range(num_epochs):
    # Forward pass
    y_pred = perceptron(X_train)
    loss = Binary_Cross_Entropy(y_pred, y_train)

    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Test the accuracy of the trained Perceptron on the testing data
with torch.no_grad():
    y_pred = perceptron(X_test)
    pred = torch.round(y_pred)
    accuracy = (pred == y_test).float().mean()
    print("Accuracy:", accuracy.item())

Accuracy: 0.9649122953414917
